# Cross Validation Index Generation

In [8]:
import pandas as pd
import numpy as np
import math
from sklearn.pipeline import Pipeline
from sklearn.model_selection import LeaveOneOut, StratifiedKFold
import pickle
import joblib
import os

## Load data

In [10]:
df_config = pd.read_csv('../data/config.csv')
train_id = 1
df_config

,hash_id,label,wav_files_path,processed_file_folder,processed_file_path,wav_files_info,cv_alg,cv_folds,cv_path,preproc_alg,...,pipeline_path,scaler_alg,train_data_path,train_trgt_path,target_label_file,target_id_file,model_path,model_inits,model_neurons,model_status
0,-7182709064797668123,ShipsEar NN Classification,../data/shipsEar_AUDIOS,../data,../data/-7182709064797668123_processed_data.csv,../data/wav_file_informations.csv,StratifiedKFolds,5,../data/indexes,Lofar,...,../data/pipelines,StandardScaler,../data/-7182709064797668123_train_data.csv,../data/-7182709064797668123_trgt_data.csv,../data/models/train_label_file.csv,../data/models/train_id_file.csv,../data/models,1,../data/models/-7182709064797668123_hidden_neu...,../data/models/-7182709064797668123_model_stat...
1,1518950927628997345,Toy Data Classification,../data/shipsEar_AUDIOS,../data,../data/1518950927628997345_processed_data.csv,../data/wav_file_informations.csv,StratifiedKFolds,5,../data/indexes,MFCC,...,../data/pipelines,StandardScaler,../data/1518950927628997345_train_data.csv,../data/1518950927628997345_trgt_data.csv,../data/models/train_label_file.csv,../data/models/train_id_file.csv,../data/models,1,../data/models/1518950927628997345_hidden_neur...,../data/models/1518950927628997345_model_statu...


### Load train data

In [11]:
df_train = pd.read_csv(df_config['train_data_path'][train_id])

In [12]:
df_train

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,target
0,-0.809787,1.140782,0.552740,1.942452,-0.099175,0.122905,0.275852,1.482409,0.423330,1.247347,...,-0.699771,0.220017,-0.916476,0.515953,0.282666,0.945631,0.277907,1.574058,0.042095,1
1,2.582713,3.161590,0.083602,-2.986270,1.151863,2.075501,-0.596150,-0.524811,1.258575,-1.426467,...,-0.218064,0.154101,0.137039,-1.850546,2.839435,0.895683,0.104921,2.725624,-0.170036,1
2,3.388922,-0.312243,0.332491,-1.109370,0.389026,-0.028511,0.292879,-3.008659,-0.940198,-2.708882,...,-0.232607,-1.320325,-0.563335,0.552087,1.852921,-0.539202,0.366420,0.859907,-0.567231,0
3,2.299062,1.340423,-0.705663,0.243180,-0.178422,-2.069673,-0.533815,-2.986325,-0.977363,-1.303734,...,-1.848854,-1.474998,-1.686704,2.852161,3.344909,-0.228653,0.191883,0.918660,-0.124210,0
4,-1.559000,-0.551521,2.361235,-2.182949,-0.161701,-0.329970,0.588126,-2.942822,0.375951,-1.165616,...,1.030261,-2.005482,0.872376,-1.896221,2.920265,1.582279,1.642354,0.126912,0.468424,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,-2.561596,-0.613549,1.281017,-2.187823,0.979110,-4.797075,0.346530,-3.235989,0.786187,1.351959,...,-0.213798,-0.179060,1.910949,-1.737410,3.314394,0.663689,0.216141,-0.913387,0.036835,0
9996,-1.181239,2.967837,0.749838,-0.779088,0.066806,-0.214362,1.716825,3.394243,-0.815834,0.818056,...,0.479844,1.490741,0.310027,0.529281,-2.758540,1.137109,0.851828,0.466088,0.422819,1
9997,0.132783,-1.728018,0.065239,-0.116372,1.932090,2.018570,0.468603,3.577870,-0.243272,0.568078,...,0.983915,-0.541892,0.684354,1.609763,-1.910008,0.140622,-2.358597,0.127640,-0.577396,3
9998,-0.073593,-0.887958,0.362963,-1.300270,-0.925996,-1.262828,0.969771,-3.227370,0.957848,-0.291534,...,-0.462273,0.119357,0.607727,-1.498298,1.331204,0.922552,0.951147,-1.333808,0.349420,3


# Stratified K-Folds

In [13]:
from sklearn.model_selection import StratifiedKFold
import pickle
import os

file_path = df_config['cv_path'][train_id]
n_folds = df_config['cv_folds'][train_id]

cv = StratifiedKFold(n_splits=n_folds)

data = df_train.drop(columns=['target']).values
target = df_train['target'].values

for idx, (trn_idx, val_idx) in enumerate(cv.split(data,target)):
    file_name = '%s_CV_fold_%i_of_%i_cv_indexes.pkl'%(df_config['hash_id'][train_id],
                                                      idx, n_folds)
    with open(os.path.join(file_path,file_name),'wb') as file_handler:
        pickle.dump([trn_idx,val_idx],file_handler)

In [14]:
data.shape

(10000, 20)

In [15]:
target.shape

(10000,)